<a href="https://colab.research.google.com/github/alfredcs/CVWorkshop17/blob/main/AI_basketball_analysis_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI basketball analysis
<img class="emoji" alt="basketabll" height="300" width="700" src="https://raw.githubusercontent.com/hardik0/AI-basketball-analysis-on-google-colab/master/static/img/analysis.gif">

<img class="emoji" alt="basketabll" height="20" width="20" src="https://github.githubassets.com/images/icons/emoji/unicode/1f3c0.png"> **Analyze basketball shots and shooting pose with machine learning!**

This is an artificial intelligence application built on the concept of object detection. Analyze basketball shots by digging into the data collected from object detection. We can get the result by simply uploading files to the web App, or submitting a POST request to the API.

**Credits:**

Special thanks to:

**Tony Chou** for this amazing github repo **[AI-basketball-analysis](https://github.com/chonyy/AI-basketball-analysis)**

**CMU Perceptual Computing Lab** for **[OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose)**

In [1]:
# First, change the runtime type to "GPU".
# Confirm runtime to GPU
! nvcc --version
! nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Fri Aug 19 16:47:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |            

In [2]:
# Openpose version tag
#ver_openpose = "v1.6.0"
# If CMake is old, Openpose build fails, so download the latest version
! cmake --version

cmake version 3.22.6

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [ ]:
#Jun 18, 2020 - Rebuild the latest CMake at that time (about 15 minutes)
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.17.3/cmake-3.17.3.tar.gz"
! tar xf cmake-3.17.3.tar.gz
! cd cmake-3.17.3 && ./configure && make && sudo make install

# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given youtube video.

## Install OpenPose


In [3]:
# Install library

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [910 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,937 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubun

In [4]:
# Clone Openpose
#! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git
! git clone https://github.com/CMU-Perceptual-Computing-Lab/openpose.git
! cd openpose && mkdir build && cd build

Cloning into 'openpose'...
remote: Enumerating objects: 16070, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 16070 (delta 0), reused 5 (delta 0), pack-reused 16064
Receiving objects: 100% (16070/16070), 84.43 MiB | 33.75 MiB/s, done.
Resolving deltas: 100% (11259/11259), done.


In [ ]:
#%rm -r openpose/build

In [5]:
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/installation.md#cmake-command-line-configuration-ubuntu-only
# Scenario 1 - Caffe not installed and OpenCV installed using apt-get
! cd openpose/build && cmake -DBUILD_PYTHON=ON .. 

# If you want to download the COCO model as well, execute the following command. (Please change "# ! cd" part to "! cd".)
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_COCO_MODEL=ON

# If you want to download the MPI model as well, execute the following command.
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_MPI_MODEL=ON

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "11.1") 
--

In [ ]:
# Openpose Building
! cd openpose/build && make -j`nproc`

In [ ]:
# Openpose Installing
! cd openpose/build && make install

In [ ]:
! echo "include /usr/local/lib" >> /etc/ld.so.conf
! ldconfig 

In [ ]:
%ls /usr/local/python

openpose/  pyopenpose.cpython-36m-x86_64-linux-gnu.so


In [ ]:
# Run and check the sample
# The output analysis result is placed under "openpose/output" from the folder icon on the left menu of the Colab screen.
! cd openpose && mkdir output && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --display 0  --write_video ./output/openpose.avi

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 22.909636 seconds.


## Detect poses on a test video

We are going to detect poses on the following youtube video:

In [ ]:
!pip install youtube-dl

In [ ]:
from IPython.display import YouTubeVideo

YOUTUBE_ID = 'RXABo9hm8B8'

YouTubeVideo(YOUTUBE_ID)

Download the above youtube video, cut the first 5 seconds and do the pose detection on that 5 seconds:

In [ ]:
!rm -rf youtube.mp4
# download the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.mp4
# detect poses on the these 5 seconds
!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

Finally, visualize the result:

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)

# <img class="emoji" alt="basketabll" height="20" width="20" src="https://github.githubassets.com/images/icons/emoji/unicode/1f3c0.png"> Analyze basketball shots and shooting pose with machine learning!

## Get a copy
Get a copy of this project by simply running the git clone command.

In [ ]:
# Original repository
#! git clone https://github.com/chonyy/AI-basketball-analysis.git

# Modified version 
! git clone https://github.com/hardik0/AI-basketball-analysis-on-google-colab
%cd AI-basketball-analysis-on-google-colab

Cloning into 'AI-basketball-analysis-on-google-colab'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 829 (delta 96), reused 130 (delta 44), pack-reused 626
Receiving objects: 100% (829/829), 292.05 MiB | 36.58 MiB/s, done.
Resolving deltas: 100% (319/319), done.
Checking out files: 100% (96/96), done.
/content/AI-basketball-analysis-on-google-colab


## Prerequisites
Before running the project, we have to install all the dependencies from requirements.txt

In [ ]:
! pip install -r colab_requirements.txt

You must restart the runtime

In [ ]:
%cd AI-basketball-analysis-on-google-colab

/content/AI-basketball-analysis-on-google-colab


## Hosting

In [ ]:
# Download ngrok for tunneling.
!if [ ! -f ./ngrok ]; then \
 wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip; \
 unzip -o ngrok-stable-linux-amd64.zip; \
 fi

In [ ]:
# Then start a mini web server.
port = 5000
!pkill ngrok
!kill $(ps x | grep -v grep | grep http.server | awk '{print $1}') 2>/dev/null

get_ipython().system_raw(
    'python app.py && python3 -m http.server {} &'
    .format(port)
)

# And, forward the port using ngrok.
get_ipython().system_raw('./ngrok http {} &'.format(port))

**Port forwarding**

Via a `ngrok` tunnel from the local machine to the internet.

In [ ]:
# Get the public address from localhost:4040 (ngrok's web interface).
import time, urllib, json
time.sleep(1)  # Give ngrok time to startup.
ngrok_data = json.load(
    urllib.request.urlopen('http://localhost:4040/api/tunnels'))
ngrok_data['tunnels'][0]['public_url']

In [ ]:
# You can connect to this external address using your mobile phone!
!pip install -q qrcode
import qrcode
qrcode.make(ngrok_data['tunnels'][0]['public_url'])

Last, get the project hosted on google colab with a single command.

To restart Server/Flask you may click Menu: **Runtime->Restart runtime**

**Note:** After restart you can start with [here](#scrollTo=xz_TJD1MgEWR)

In [ ]:
# First run this cell then open above link 
!python app.py

Or you can run below cell 

In [ ]:
import os
import sys
import cv2

from flask import Flask, render_template, Response,  request, session, redirect, url_for, send_from_directory, flash, jsonify, abort
from werkzeug.utils import secure_filename
from PIL import Image


from src.config import shooting_result
from src.app_helper import getVideoStream, get_image, detectionAPI

app = Flask(__name__)

UPLOAD_FOLDER = './static/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
#useless key, in order to use session
app.secret_key = "super secret key" 

@app.route("/")
def index():
    return render_template("index.html")

@app.route('/detection_json', methods=['GET', 'POST'])
def detection_json():
    if request.method == 'POST':
        response = []
        f = request.files['image']
        # create a secure filename
        filename = secure_filename(f.filename)
        print("filename", filename)
        # save file to /static/uploads
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        print("filepath", filepath)
        f.save(filepath)
        detectionAPI(response, filepath)
        print(response)
        try:
            return jsonify(response), 200
        except FileNotFoundError:
            abort(404)


@app.route('/sample_detection', methods=['GET', 'POST'])
def upload_sample_image():
    if request.method == 'POST':
        response = []
        filename = "sample_image.jpg"
        print("filename", filename)
        filepath = "./static/uploads/sample_image.jpg"
        print("filepath", filepath)
        get_image(filepath, filename, response)
        return render_template("shot_detection.html", display_detection=filename, fname=filename, response=response)

@app.route('/basketball_detection', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        response = []
        f = request.files['image']
        # create a secure filename
        filename = secure_filename(f.filename)
        print("filename", filename)
        # save file to /static/uploads
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        print("filepath", filepath)
        f.save(filepath)
        get_image(filepath, filename, response)
        return render_template("shot_detection.html", display_detection=filename, fname=filename, response=response)

@app.route('/sample_analysis', methods=['GET', 'POST'])
def upload_video():
    global shooting_result
    shooting_result['attempts'] = 0
    shooting_result['made'] = 0
    shooting_result['miss'] = 0
    if (os.path.exists("./static/detections/trajectory_fitting.jpg")):
        os.remove("./static/detections/trajectory_fitting.jpg")
    if request.method == 'POST':
        filename = "sample_video.mp4"
        print("filename", filename)
        filepath = "./static/uploads/sample_video.mp4"
        print("filepath", filepath)
        session['video_path'] = filepath
        return render_template("shooting_analysis.html")

@app.route('/shooting_analysis', methods=['GET', 'POST'])
def upload_sample_video():
    global shooting_result
    shooting_result['attempts'] = 0
    shooting_result['made'] = 0
    shooting_result['miss'] = 0
    if (os.path.exists("./static/detections/trajectory_fitting.jpg")):
        os.remove("./static/detections/trajectory_fitting.jpg")
    if request.method == 'POST':
        f = request.files['video']
        # create a secure filename
        filename = secure_filename(f.filename)
        print("filename", filename)
        # save file to /static/uploads
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        print("filepath", filepath)
        f.save(filepath)
        session['video_path'] = filepath
        return render_template("shooting_analysis.html")

@app.route('/video_feed')
def video_feed():
    video_path = session.get('video_path', None)
    stream = getVideoStream(video_path)
    return Response(stream,
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route("/result", methods=['GET', 'POST'])
def result():
    return render_template("result.html", shooting_result=shooting_result)

#disable caching
@app.after_request
def after_request(response):
    response.headers["Cache-Control"] = "no-cache, no-store, must-revalidate, public, max-age=0"
    response.headers["Expires"] = 0
    response.headers["Pragma"] = "no-cache"
    return response

if __name__ == '__main__':
    app.run(debug=True, use_reloader=True)